# Pandas 百问百答 之 自定义排序
## Python 大咖谈 ~ 呆鸟出品

1. 自定义索引排序
2. 自定义列值排序
3. 利用辅助列实现自定义列值排序
4. 自定义列序

In [1]:
import pandas as pd

In [2]:
sales = pd.read_excel("data/自定义排序.xlsx","销售数据",index_col="序号")
sales

,分公司,销售额,门店
序号,,,
1,上海,26677,上海一店
2,广州,16544,广州二店
3,深圳,15655,深圳二店
4,北京,36986,北京一店
5,上海,18923,上海二店
6,深圳,44161,深圳一店
7,广州,26409,广州一店
8,北京,93223,北京二店
9,北京,56586,北京三店


不论按升序，还是按降序排列，sort_values() 都不能实现我们想要的，按北京、上海、广州这样的自定义排序。

In [3]:
sales.sort_values(by=["分公司","门店"],ascending=True)

,分公司,销售额,门店
序号,,,
1,上海,26677,上海一店
5,上海,18923,上海二店
4,北京,36986,北京一店
9,北京,56586,北京三店
8,北京,93223,北京二店
7,广州,26409,广州一店
2,广州,16544,广州二店
6,深圳,44161,深圳一店
3,深圳,15655,深圳二店


### 自定义索引排序

要是数据量不大，可以尝试一下自定义索引排序。

In [4]:
reorder = [4,8,9,1,5,7,2,6,3]

sales.reindex(reorder)

,分公司,销售额,门店
序号,,,
4,北京,36986,北京一店
8,北京,93223,北京二店
9,北京,56586,北京三店
1,上海,26677,上海一店
5,上海,18923,上海二店
7,广州,26409,广州一店
2,广州,16544,广州二店
6,深圳,44161,深圳一店
3,深圳,15655,深圳二店


虽然只有 9 条数据，但这种一个一个写索引，呆鸟已经看的头晕眼花了，这要再多点，更受不了了，显然，数据量稍微多一些，这种方式就不适合了。

### 自定义列值排序

下面，我们学习一种更简单，更直观的方式。

这里的原理是把列转换为类别型，通过指定**类别排序**，按列值排序。

In [5]:
sales['分公司'] = pd.Categorical(sales['分公司'], categories=[
                              '北京', '上海', '广州', '深圳'], ordered=True)

sales.sort_values(by='分公司')

,分公司,销售额,门店
序号,,,
4,北京,36986,北京一店
8,北京,93223,北京二店
9,北京,56586,北京三店
1,上海,26677,上海一店
5,上海,18923,上海二店
2,广州,16544,广州二店
7,广州,26409,广州一店
3,深圳,15655,深圳二店
6,深圳,44161,深圳一店


这样很方便地就实现了按分公司排列，但并未实现门店列的排序，比如，广州与深圳的二店排在了一店前面。

虽然，也可以把门店列改为类别型，但门店很多就不直观了，因此，我们再引入一种方式。

同时，也请注意，类似这种分公司与门店名称的排序，要在设计数据库时提供分公司与门店代码，在数据导出时也要导出相应代码，代码依据规律创建，在排序时就可以用上。

如果代码规律与排序规律还是不一致，就只好自行维护一套自定义规律表了。

### 利用辅助列实现自定义列值排序

首先，要导入自定义的分公司与门店代码表

In [6]:
branch = pd.read_excel('data/自定义排序.xlsx','分公司代码')
branch

,分公司代码,分公司
0,Branch001,北京
1,Branch002,上海
2,Branch003,广州
3,Branch004,深圳


In [7]:
department = pd.read_excel('data/自定义排序.xlsx','门店代码')
department

,门店代码,门店
0,bj_001,北京一店
1,bj_002,北京二店
2,bj_003,北京三店
3,sh_001,上海一店
4,sh_002,上海二店
5,gz_001,广州一店
6,gz_002,广州二店
7,sz_001,深圳一店
8,sz_002,深圳二店


In [8]:
sales = pd.merge(sales, branch, on="分公司", how="left")
sales = pd.merge(sales, department, on="门店",how="left")
sales

,分公司,销售额,门店,分公司代码,门店代码
0,上海,26677,上海一店,Branch002,sh_001
1,广州,16544,广州二店,Branch003,gz_002
2,深圳,15655,深圳二店,Branch004,sz_002
3,北京,36986,北京一店,Branch001,bj_001
4,上海,18923,上海二店,Branch002,sh_002
5,深圳,44161,深圳一店,Branch004,sz_001
6,广州,26409,广州一店,Branch003,gz_001
7,北京,93223,北京二店,Branch001,bj_002
8,北京,56586,北京三店,Branch001,bj_003


现在，用 `sort_values()`，就可以实现自定义排序了。

In [9]:
sales.sort_values(['分公司代码','门店代码'],inplace=True)
sales

,分公司,销售额,门店,分公司代码,门店代码
3,北京,36986,北京一店,Branch001,bj_001
7,北京,93223,北京二店,Branch001,bj_002
8,北京,56586,北京三店,Branch001,bj_003
0,上海,26677,上海一店,Branch002,sh_001
4,上海,18923,上海二店,Branch002,sh_002
6,广州,26409,广州一店,Branch003,gz_001
1,广州,16544,广州二店,Branch003,gz_002
5,深圳,44161,深圳一店,Branch004,sz_001
2,深圳,15655,深圳二店,Branch004,sz_002


### 自定义列序

到目前为止，列序有些乱，严总看了，肯定又会批评不注意细节了，所以还要按以下顺序重排列序：分公司代码、分公司、门店代码、门店、销售额。

In [10]:
sales.sort_index(axis='columns',ascending=True)

,分公司,分公司代码,销售额,门店,门店代码
3,北京,Branch001,36986,北京一店,bj_001
7,北京,Branch001,93223,北京二店,bj_002
8,北京,Branch001,56586,北京三店,bj_003
0,上海,Branch002,26677,上海一店,sh_001
4,上海,Branch002,18923,上海二店,sh_002
6,广州,Branch003,26409,广州一店,gz_001
1,广州,Branch003,16544,广州二店,gz_002
5,深圳,Branch004,44161,深圳一店,sz_001
2,深圳,Branch004,15655,深圳二店,sz_002


从上面的列子可以看到，`sort_index()` 只支持按字母或数字排序，不支持自定义列排序，咱们还得想办法。

In [11]:
sales = sales[['分公司代码', '分公司', '门店代码', '门店', '销售额']]
sales

,分公司代码,分公司,门店代码,门店,销售额
3,Branch001,北京,bj_001,北京一店,36986
7,Branch001,北京,bj_002,北京二店,93223
8,Branch001,北京,bj_003,北京三店,56586
0,Branch002,上海,sh_001,上海一店,26677
4,Branch002,上海,sh_002,上海二店,18923
6,Branch003,广州,gz_001,广州一店,26409
1,Branch003,广州,gz_002,广州二店,16544
5,Branch004,深圳,sz_001,深圳一店,44161
2,Branch004,深圳,sz_002,深圳二店,15655


至此，列序也排好了，但这就完了吗？

如果这样给严总，**肯定还会挨批！**

为什么呢？

大家看下 index， 也就是索引序号，是不是乱序的？另外对严总来说，他并不关心这些代码，对他来说只是冗余信息，影响他看数据的效率与效果，所以也应该去掉。

In [12]:
# 删掉没用的列
del sales['分公司代码'], sales['门店代码']

In [13]:
# 重置索引
sales.reset_index(drop=True,inplace=True)
sales

,分公司,门店,销售额
0,北京,北京一店,36986
1,北京,北京二店,93223
2,北京,北京三店,56586
3,上海,上海一店,26677
4,上海,上海二店,18923
5,广州,广州一店,26409
6,广州,广州二店,16544
7,深圳,深圳一店,44161
8,深圳,深圳二店,15655
